In [1]:
#importing libraries and data
import random
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [3]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [4]:
text = open(filepath, 'rb').read().decode(encoding = 'utf-8').lower() #read text file into a variable

In [5]:
text = text[300000:800000] #setting a training set

In [6]:
characters = sorted(set(text)) #get sorted list of unique characters in the text file

In [7]:
char_to_index = dict((c,i) for i,c in enumerate(characters)) #char as key, number as value

In [8]:
index_to_char = dict((i,c) for i,c in enumerate(characters)) #number as key, char as value

In [9]:
SEQ_LENGTH = 40
STEP_SIZE = 3

In [10]:
sentences = [] #features
next_characters = [] #target

In [11]:
#To retrive sentences from the text file into training data
for i in range (0,len(text) - SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i : i + SEQ_LENGTH])
  next_characters.append(text[i + SEQ_LENGTH])

In [16]:
#convert to boolean data for NN model
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)
y = np.zeros((len(sentences),len(characters)), dtype=bool)

In [17]:
for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
    x[i, t, char_to_index[character]] = 1
  y[i, char_to_index[next_characters[i]]] = 1

In [18]:
#Building the NN Model
model = Sequential()
model.add(LSTM(128, input_shape = (SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(lr=0.01))
model.fit(x, y, batch_size = 256, epochs = 4)
model.save('textgenerator.model')

Epoch 1/4
651/651 [==============================] - 118s 178ms/step - loss: 2.7135
Epoch 2/4
651/651 [==============================] - 118s 181ms/step - loss: 2.3138
Epoch 3/4
651/651 [==============================] - 114s 175ms/step - loss: 2.2010
Epoch 4/4
651/651 [==============================] - 116s 178ms/step - loss: 2.1231


In [19]:
saved_model = tf.keras.models.load_model('textgenerator.model')

In [26]:
#Function that takes probability predictions from model and chooses one option.Taken from the official keras tutorial
def sample(preds, temperature = 1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature # temperature mandates the probability predictions. Higher temperature -> more creativity
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [24]:
#Starting text from shaksphere (first 40 characters) The text that will be generated after this will be the models creation
def generate_text(length, temperature):
  start_index = random.randint(0, len(text)- SEQ_LENGTH - 1)
  generated = ''
  start_sentence = text[start_index : start_index + SEQ_LENGTH]
  generated += start_sentence
  for i in range(length):
    x = np.zeros((1, SEQ_LENGTH, len(characters)))
    for t, character in enumerate(start_sentence):
      x[0, t, char_to_index[character]] = 1

    predictions = model.predict(x, verbose = 0)[0]
    next_index = sample(predictions, temperature)
    next_character = index_to_char[next_index]

    generated += next_character
    start_sentence = start_sentence[1:] + next_character

  return generated

In [22]:
#Now, we try outputting some sentences with varying levels of temperature. temperature is proportional to creativity.

In [32]:
print('- - - - - 0.2 - - - - -')

- - - - - 0.2 - - - - -


In [27]:
#temperature = 0.2
print (generate_text(300, 0.2))

s, set upon our foes
our ancient word of the her sore the with the hard the sure the sord the the the hard the king and and the mart the mert and my the lord the proust the will be the with the sour the here the will the ward the will the the sore sors mand be the will the the with the hard the beat the mand the cores and and but me the s


In [33]:
print('- - - - - 0.4 - - - - -')

- - - - - 0.4 - - - - -


In [28]:
#temperature = 0.4
print (generate_text(300, 0.4))

iege,
pardon me, if you please; if not, my dond reans my the ring of our mand with me maren shar word in shand her an and sthe for the merser hat me sore the beclond man the hat the sin thou wall the wist be to that and reat of searn the pronger with and and the me that with the the cond her proment richord for the self il the freck no me


In [34]:
print('- - - - - 0.6 - - - - -')

- - - - - 0.6 - - - - -


In [29]:
#temperature = 0.6
print (generate_text(300, 0.6))

t dropp'd down yet.

first lord:
the highe fard smer sor this sorin

apricking the sing mum and and watherfire theren thy mave
romy manc:
in of the mend to comest bet mist the king the her sann me lpateru? and son y more's mingen and in shis thes bland beny stall the romling to the to the ou menger mand and hart shen in ant un the kence g


In [35]:
print('- - - - - 0.8 - - - - -')

- - - - - 0.8 - - - - -


In [30]:
#temperature = 0.8
print (generate_text(300, 0.8))

adversary,
and yours, close fighting ered this sangr-ingor be,
and ix comenstanct now:
to noy armay
the butd be the rintt liomn's not.

lavesol foll it not anle whill have rence than woots nontor ale thall switl oul lenges mingiry,
list lang and thaug nom frmarde the bu y the nortetor'd in me ty ur tous ond;
in stert for my mingrangy and 


In [36]:
print('- - - - - 1.0 - - - - -')

- - - - - 1.0 - - - - -


In [31]:
#temperature = 1.0
print (generate_text(300, 1))

vered with a groan,
'o, farewell, warwick, ay plisitt and,
nor glloin ferinnt stlluse broge ed, the tient.

heroon
on sive;
ind't sherfore?

pprcadne,
ma may herse;
ond ofst and that
mofr wunt nosh lo, cemackint mo ant gor enrthe,
and cooc stay,
a parime.

guruek:
th sicllive ane so to bes,
tay grey?

for pmy,
yor ih may wailn,
to i gone!


In [37]:
print (generate_text(300, 0.29))

ng the principal of all the deer.

secone:
i hard me praint:
and and will the but the soust thou sard the his me the greand and my but the ward in the here with the his the pringer beand but the sour and the promes and my prave the the mand and shard and the sore the king the mand that me hard are and and and and the mand and the ronge fr
